## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from keras.layers import BatchNormalization, Activation
from keras.utils import np_utils
from keras.optimizers import Adam
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
np.set_printoptions(suppress=True)

trainD = np.load("/home/hsiehch/30s/2D_spect/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/2D_spect/validate_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/2D_spect/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainD = np.append(trainD, validationD, axis=0)
trainL = np.append(trainL, validationL, axis=0)
trainD = np.append(trainD, testD, axis=0)
trainL = np.append(trainL, testL, axis=0)

data = trainD.reshape((trainD.shape[0], trainD.shape[1], trainD.shape[2], 1))
labels = np_utils.to_categorical(trainL, 4)

# data, labels = shuffle(trainData, trainLabel, random_state=0)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

KFold(n_splits=5, random_state=None, shuffle=True)


In [3]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(learning_rate=0):
    model = Sequential() 
    model.add(Conv2D(filters = 32, kernel_size = 3, data_format="channels_last", input_shape = (data.shape[1], data.shape[2], 1)))
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 32, kernel_size = 3, activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu"))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(Dense(160, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(4, activation = "softmax"))
    
    adam = Adam(lr=learning_rate)
    model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
    
    return model

batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]
parameter_grid = dict(batch_size=batch_S, learning_rate=learning_rate)
model = KerasClassifier(build_fn = create_model, epochs=80, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, cv=kf, verbose = 1)
grid_result = grid.fit(data, labels)

print('Finish training!')

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Epoch 1/80
8120/8120 [==============================] - 9s 1ms/step - loss: 1.1100 - acc: 0.5677
Epoch 2/80
8120/8120 [==============================] - 5s 655us/step - loss: 1.0285 - acc: 0.5708
Epoch 3/80
8120/8120 [==============================] - 5s 645us/step - loss: 1.0185 - acc: 0.5780
Epoch 4/80
8120/8120 [==============================] - 5s 662us/step - loss: 1.0169 - acc: 0.5794
Epoch 5/80
8120/8120 [==============================] - 5s 648us/step - loss: 1.0148 - acc: 0.5817
Epoch 6/80
8120/8120 [==============================] - 5s 652us/step - loss: 1.0119 - acc: 0.5824
Epoch 7/80
8120/8120 [==============================] - 5s 667us/step - loss: 1.0059 - acc: 0.5820
Epoch 8/80
8120/8120 [==============================] - 5s 654us/step - loss: 1.0067 - acc: 0.5831
Epoch 9/80
8120/8120 [==============================] - 5s 656us/step - loss: 1.0076 - acc: 0.5829
Epoch 10/80
8120/8120 [==========================

8121/8121 [==============================] - 7s 817us/step - loss: 1.0914 - acc: 0.5833
Epoch 2/80
8121/8121 [==============================] - 6s 683us/step - loss: 1.0151 - acc: 0.5861
Epoch 3/80
8121/8121 [==============================] - 6s 681us/step - loss: 1.0065 - acc: 0.5868
Epoch 4/80
8121/8121 [==============================] - 5s 667us/step - loss: 1.0066 - acc: 0.5877
Epoch 5/80
8121/8121 [==============================] - 6s 682us/step - loss: 1.0073 - acc: 0.5874
Epoch 6/80
8121/8121 [==============================] - 6s 680us/step - loss: 1.0065 - acc: 0.5877
Epoch 7/80
8121/8121 [==============================] - 6s 685us/step - loss: 1.0058 - acc: 0.5877
Epoch 8/80
8121/8121 [==============================] - 5s 663us/step - loss: 1.0066 - acc: 0.5879
Epoch 9/80
8121/8121 [==============================] - 6s 688us/step - loss: 0.9879 - acc: 0.5877
Epoch 10/80
8121/8121 [==============================] - 6s 680us/step - loss: 0.9781 - acc: 0.5885
Epoch 11/80
8121/812

8121/8121 [==============================] - 6s 776us/step - loss: 1.1214 - acc: 0.5595
Epoch 2/80
8121/8121 [==============================] - 5s 675us/step - loss: 1.0212 - acc: 0.5755
Epoch 3/80
8121/8121 [==============================] - 6s 682us/step - loss: 1.0177 - acc: 0.5776
Epoch 4/80
8121/8121 [==============================] - 6s 679us/step - loss: 1.0105 - acc: 0.5808
Epoch 5/80
8121/8121 [==============================] - 5s 671us/step - loss: 1.0098 - acc: 0.5815
Epoch 6/80
8121/8121 [==============================] - 5s 667us/step - loss: 1.0108 - acc: 0.5851
Epoch 7/80
8121/8121 [==============================] - 6s 702us/step - loss: 1.0057 - acc: 0.5832
Epoch 8/80
8121/8121 [==============================] - 5s 677us/step - loss: 1.0080 - acc: 0.5869
Epoch 9/80
8121/8121 [==============================] - 5s 664us/step - loss: 1.0032 - acc: 0.5864
Epoch 10/80
8121/8121 [==============================] - 6s 682us/step - loss: 0.9874 - acc: 0.5864
Epoch 11/80
8121/812

Epoch 2/80
8121/8121 [==============================] - 6s 679us/step - loss: 1.0142 - acc: 0.5838
Epoch 3/80
8121/8121 [==============================] - 5s 671us/step - loss: 1.0107 - acc: 0.5865
Epoch 4/80
8121/8121 [==============================] - 5s 670us/step - loss: 1.0083 - acc: 0.5868
Epoch 5/80
8121/8121 [==============================] - 6s 696us/step - loss: 1.0057 - acc: 0.5876
Epoch 6/80
8121/8121 [==============================] - 5s 672us/step - loss: 1.0068 - acc: 0.5882
Epoch 7/80
8121/8121 [==============================] - 6s 680us/step - loss: 1.0078 - acc: 0.5876
Epoch 8/80
8121/8121 [==============================] - 6s 696us/step - loss: 1.0020 - acc: 0.5881
Epoch 9/80
8121/8121 [==============================] - 5s 667us/step - loss: 1.0047 - acc: 0.5880
Epoch 10/80
8121/8121 [==============================] - 6s 698us/step - loss: 0.9970 - acc: 0.5876
Epoch 11/80
8121/8121 [==============================] - 6s 696us/step - loss: 0.9802 - acc: 0.5877
Epoch 12

8121/8121 [==============================] - 6s 685us/step - loss: 1.0200 - acc: 0.5810
Epoch 3/80
8121/8121 [==============================] - 5s 673us/step - loss: 1.0182 - acc: 0.5834
Epoch 4/80
8121/8121 [==============================] - 5s 677us/step - loss: 1.0085 - acc: 0.5861
Epoch 5/80
8121/8121 [==============================] - 6s 684us/step - loss: 1.0091 - acc: 0.5875
Epoch 6/80
8121/8121 [==============================] - 6s 681us/step - loss: 1.0074 - acc: 0.5865
Epoch 7/80
8121/8121 [==============================] - 5s 672us/step - loss: 1.0063 - acc: 0.5871
Epoch 8/80
8121/8121 [==============================] - 6s 685us/step - loss: 1.0048 - acc: 0.5870
Epoch 9/80
8121/8121 [==============================] - 6s 689us/step - loss: 1.0039 - acc: 0.5869
Epoch 10/80
8121/8121 [==============================] - 5s 676us/step - loss: 0.9935 - acc: 0.5864
Epoch 11/80
8121/8121 [==============================] - 6s 681us/step - loss: 0.9787 - acc: 0.5881
Epoch 12/80
8121/81

...

8121/8121 [==============================] - 5s 571us/step - loss: 0.9940 - acc: 0.5876
Epoch 61/80
8121/8121 [==============================] - 5s 565us/step - loss: 0.9939 - acc: 0.5876
Epoch 62/80
8121/8121 [==============================] - 5s 558us/step - loss: 0.9948 - acc: 0.5876
Epoch 63/80
8121/8121 [==============================] - 4s 543us/step - loss: 0.9940 - acc: 0.5876
Epoch 64/80
8121/8121 [==============================] - 5s 559us/step - loss: 0.9938 - acc: 0.5876
Epoch 65/80
8121/8121 [==============================] - 4s 540us/step - loss: 0.9939 - acc: 0.5876
Epoch 66/80
8121/8121 [==============================] - 4s 548us/step - loss: 0.9939 - acc: 0.5876
Epoch 67/80
8121/8121 [==============================] - 4s 540us/step - loss: 0.9939 - acc: 0.5876
Epoch 68/80
8121/8121 [==============================] - 4s 548us/step - loss: 0.9937 - acc: 0.5876
Epoch 69/80
8121/8121 [==============================] - 4s 539us/step - loss: 0.9942 - acc: 0.5876
Epoch 70/80


Epoch 61/80
8121/8121 [==============================] - 4s 541us/step - loss: 0.9913 - acc: 0.5880
Epoch 62/80
8121/8121 [==============================] - 4s 542us/step - loss: 0.9911 - acc: 0.5880
Epoch 63/80
8121/8121 [==============================] - 4s 537us/step - loss: 0.9916 - acc: 0.5880
Epoch 64/80
8121/8121 [==============================] - 4s 537us/step - loss: 0.9911 - acc: 0.5880
Epoch 65/80
8121/8121 [==============================] - 4s 543us/step - loss: 0.9914 - acc: 0.5880
Epoch 66/80
8121/8121 [==============================] - 4s 547us/step - loss: 0.9912 - acc: 0.5880
Epoch 67/80
8121/8121 [==============================] - 5s 560us/step - loss: 0.9915 - acc: 0.5880
Epoch 68/80
8121/8121 [==============================] - 5s 569us/step - loss: 0.9911 - acc: 0.5880
Epoch 69/80
8121/8121 [==============================] - 5s 560us/step - loss: 0.9912 - acc: 0.5880
Epoch 70/80
8121/8121 [==============================] - 4s 551us/step - loss: 0.9914 - acc: 0.5880


8121/8121 [==============================] - 4s 544us/step - loss: 0.9936 - acc: 0.5870
Epoch 62/80
8121/8121 [==============================] - 4s 553us/step - loss: 0.9930 - acc: 0.5870
Epoch 63/80
8121/8121 [==============================] - 5s 559us/step - loss: 0.9936 - acc: 0.5870
Epoch 64/80
8121/8121 [==============================] - 5s 562us/step - loss: 0.9934 - acc: 0.5870
Epoch 65/80
8121/8121 [==============================] - 4s 543us/step - loss: 0.9934 - acc: 0.5870
Epoch 66/80
8121/8121 [==============================] - 4s 548us/step - loss: 0.9933 - acc: 0.5870
Epoch 67/80
8121/8121 [==============================] - 4s 544us/step - loss: 0.9935 - acc: 0.5870
Epoch 68/80
8121/8121 [==============================] - 4s 543us/step - loss: 0.9935 - acc: 0.5870
Epoch 69/80
8121/8121 [==============================] - 4s 548us/step - loss: 0.9934 - acc: 0.5870
Epoch 70/80
8121/8121 [==============================] - 4s 544us/step - loss: 0.9932 - acc: 0.5870
Epoch 71/80


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 854.2min finished


Epoch 1/80
10151/10151 [==============================] - 25s 2ms/step - loss: 1.0386 - acc: 0.5840
Epoch 2/80
10151/10151 [==============================] - 6s 544us/step - loss: 1.0016 - acc: 0.5870
Epoch 3/80
10151/10151 [==============================] - 6s 571us/step - loss: 0.9884 - acc: 0.5858
Epoch 4/80
10151/10151 [==============================] - 6s 542us/step - loss: 0.9777 - acc: 0.5884
Epoch 5/80
10151/10151 [==============================] - 5s 542us/step - loss: 0.9675 - acc: 0.5877
Epoch 6/80
10151/10151 [==============================] - 6s 547us/step - loss: 0.9658 - acc: 0.5880
Epoch 7/80
10151/10151 [==============================] - 6s 571us/step - loss: 0.9641 - acc: 0.5886
Epoch 8/80
10151/10151 [==============================] - 5s 542us/step - loss: 0.9616 - acc: 0.5904
Epoch 9/80
10151/10151 [==============================] - 6s 547us/step - loss: 0.9600 - acc: 0.5891
Epoch 10/80
10151/10151 [==============================] - 6s 543us/step - loss: 0.9579 - ac

In [4]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean:%f (STD:%f) with: %r" % (mean, stdev, param))

Mean:0.689587 (STD:0.015202) with: {'batch_size': 30, 'learning_rate': 5e-05}
Mean:0.740715 (STD:0.007300) with: {'batch_size': 30, 'learning_rate': 0.0001}
Mean:0.767905 (STD:0.013609) with: {'batch_size': 30, 'learning_rate': 0.0005}
Mean:0.611368 (STD:0.053780) with: {'batch_size': 30, 'learning_rate': 0.001}
Mean:0.587036 (STD:0.005333) with: {'batch_size': 30, 'learning_rate': 0.005}
Mean:0.639149 (STD:0.019213) with: {'batch_size': 50, 'learning_rate': 5e-05}
Mean:0.724362 (STD:0.030142) with: {'batch_size': 50, 'learning_rate': 0.0001}
Mean:0.762289 (STD:0.023322) with: {'batch_size': 50, 'learning_rate': 0.0005}
Mean:0.655600 (STD:0.061955) with: {'batch_size': 50, 'learning_rate': 0.001}
Mean:0.587036 (STD:0.005333) with: {'batch_size': 50, 'learning_rate': 0.005}
Mean:0.633041 (STD:0.027997) with: {'batch_size': 70, 'learning_rate': 5e-05}
Mean:0.686730 (STD:0.027745) with: {'batch_size': 70, 'learning_rate': 0.0001}
Mean:0.737464 (STD:0.022575) with: {'batch_size': 70, 'lear

In [5]:
grid_result.best_score_

0.7681016635277061

In [6]:
grid_result.best_params_

{'batch_size': 120, 'learning_rate': 0.001}